# Bayes - A Nonparametric Bayesian Approach to Modeling Overlapping Clusters

In [1]:
%matplotlib inline

In [2]:
from matplotlib.pyplot import figure, show
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal, bernoulli, beta
import random

#### Get movies data from MovieLens

The clusters matrix is Z
The binary matrix is X

In [3]:
Z = pd.read_csv('clusters_matrix.csv', sep=',', index_col=0)
X = pd.read_csv('binary_data_matrix.csv', sep=',', index_col=0)
#X.columns = X.columns.astype(int)
#Z.columns = Z.columns.astype(int)
X = X.as_matrix()
Z = Z.as_matrix()

In [4]:
def clear_users(data):    
    nbr_clear = 0
    list_col = []
    for i in range(data.shape[1]):
        temp_ratings = data[:,i]
        nbr_ratings = sum(temp_ratings)    
        if nbr_ratings > 20: 
            list_col.append(i)
            #data = np.delete(data, i, 1)
        else:
            nbr_clear += 1
    if len(list_col)>0:
        data_cleaned = data[:,list_col]
    else:
        data_cleaned = data
    return data_cleaned, nbr_clear

def clear_movies(data):
    nbr_clear = 0
    list_row = []
    for i in range(data.shape[0]):
        temp_ratings = data[i,:]
        nbr_ratings = sum(temp_ratings)    
        if nbr_ratings > 10: 
            list_row.append(i)
            #data = np.delete(data, i, 0)
        else:
            nbr_clear += 1
    if len(list_row)>0:
        data_cleaned = data[list_row,:]
    else:
        data_cleaned = data
    return data_cleaned, nbr_clear

In [5]:
nbr_clear = 1
X = X[0:650]

while nbr_clear > 0:
    X, nbr_clear = clear_movies(X)
    X, nbr_clear = clear_users(X)

X = X[0:500] 
Z = Z[0:500]

In [6]:
X.shape

(500, 746)

In [7]:
Z.shape

(500, 18)

In [8]:
def get_mat_random(mat_x, list_rows):
    mat_y = np.zeros((mat_x.shape[0], mat_x.shape[1]))
    for i in range(mat_x.shape[0]):
        mat_y[i,:] = mat_x[list_rows[i],:]
    return mat_y

In [9]:
# Need to randomize the matrix in order to have efficient train et test datasets
list_random_rows = random.sample(range(500), 500)
X = get_mat_random(X, list_random_rows)
Z = get_mat_random(Z, list_random_rows)

#### Initialization of all parameters

Our data are a binary matrix : we can use a multivariate Bernouilli simulation

In [10]:
def m_without_i_k(Z, i, k):
    return (sum(Z[:,k])-Z[i,k])

def likelihood_bern(X,Z,theta,i, c):  #not normalized
    lh=1
    for d in range(D):
        num=1
        denom=1
        for k in range(K):
            num = num * theta[k,d]**Z[i,k]
            denom = denom * (1-theta[k,d])**Z[i,k]
        theta_tilde_d = num/(denom+num)
        lh = lh * bernoulli.pmf(X[i,d],theta_tilde_d)    
    return lh

In [19]:
NumIters = 1
N = X.shape[0]
K = Z.shape[1]
D = X.shape[1]

Z_hat = np.zeros((N,K))  #Matrix of clusters [observations]*[# clusters - takes 1 if belongs to cluster]
PZ_hat = np.zeros((N,K)) #Matrix of cluster probabilities
theta = np.empty([K,D])
np.random.seed(1234)

alpha = round(sum(Z.T).mean()) #alphe controles the expected number of clusters a datapoint will belong to
c = sum(likelihood_bern(X, Z, theta, [i for i in range(N)], 1) )

for i in range(K):
    for j in range(D):
        u=np.random.beta(alpha/K, 1)
        theta[i,j]=u

#On prend les n premieres observations et on leur donne les bons clusters associés pour entrainer le modele
n=200
Z_hat[0:n-1,]=Z[0:n-1,]

In [20]:
def transition(theta, theta_known, omega):
    return beta.pdf(theta, omega*theta_known, omega*(1-theta_known))

def proba_xd(Z,X_d,d,k,K,theta,theta_kd=None):
    proba_xd = 1
    new_theta = theta
    if theta_kd:
        new_theta[k,d]=theta_kd
    for i in range(X_d.shape[0]):
        num=1
        denom=1
        for k in range(K):
            num = num * new_theta[k,d]**Z[i,k]
            denom = denom * (1-new_theta[k,d])**Z[i,k]
        theta_tilde_d = num/(denom+num)
        proba_xd= proba_xd * bernoulli.pmf(X[i,d],theta_tilde_d) 
    return proba_xd

In [21]:
Z[1,]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [23]:
%%time
iterations = 5
for j in range(iterations):
    print("j : " + str(j) + "/" + str(iterations-1))
    for i in range(n,N,1):
        #print(str(i) + "/" + str(N-1))
        k_plus = [] #k+ is the number of clusters which data points, excluding i, belong to
        for k_ in range(K):
            if Z_hat[i,k_] == 0:
                k_plus.append(k_)  #for each obs, if proba to belong to cluster k_ is null, add it to k_plus
                                   #for data not in the training set, k_plus will take all possible values in 0...K

        for k in k_plus:
            if Z_hat[i,k] == 0: #exclude data in the training set for which we already have the true categories
                #z_ik ⇠ zik|z−i,k, xi,theta
                Z_hat[i,k] = 1 #Set Z_hat to one (proposal)
                #compute bernouilli likelihood (not normalized)
                lh_bern=likelihood_bern(X,Z_hat,theta,i,c)
                #compute matrix of probas of Z
                PZ_hat[i,k] = (m_without_i_k(Z_hat, i, k)/N)*lh_bern
                
                Z_hat[i,k] = 0  #reset Z_hat to zero
                u = np.random.beta(alpha/K, 1)  #PZ_hat is not normalized to [0,1]
                
                #print('Pz=', PZ_hat[i,k], 'u =', u, 'i=',i, 'k=',k)
                
                #Propose adding new clusters 
                #Accept or reject proposal
                if u<PZ_hat[i,k]:
                    Z_hat[i,k]=1 
                    print('Pz=', PZ_hat[i,k], 'u =', u, 'i=',i, 'k=',k)
        
    #Resample theta|Z,X using MH proposal
    print("Resampling")
    prob_A=0
    omega=0.5
    for k in range(K):
        #print("k :" + str(k) + "/" + str(K-1))
        for d in range(D):
            #print("d :" + str(d) + "/" + str(D-1))
            
            #generate proposal theta based on Beta(omega*theta,omega*(1-theta))
            theta_prop=beta.rvs(omega*theta[k,d],omega*(1-theta[k,d]))
            print('theta_kd =', theta[k,d], 
                'theta_prop =', theta_prop)
            #T(theta_kd|theta'_kd,omega)
            transition_num = transition(theta_prop, theta[k,d], omega)
            print('transition_num =', transition_num)
            #T(theta'_kd|theta_kd,omega)
            transition_denom = transition(theta[k,d], theta_prop, omega)
            print('transition_denom =', transition_denom)
            
            #p(theta'_d)
            p_theta_num = beta.pdf(theta_prop, alpha/K, 1)#ici j'utilise theta'_kd et non theta'_d > TODO : voir comment calculer Theta'd
            print('p_theta_num =',p_theta_num)
            #p(theta_d)
            p_theta_denom = beta.pdf(theta[k,d], alpha/K, 1)
            print('p_theta_denom =', p_theta_denom)
            
            #p(x_d|theta'_kd,Z)
            p_xd_num = proba_xd(Z,X[:,d],d,k,K,theta,theta_prop)
            print('p_xd_num =', p_xd_num)
            #p(x_d|theta_kd,Z)
            p_xd_denom = proba_xd(Z,X[:,d],d,k,K,theta,None)
            print('p_xd_denom =', p_xd_denom)
            
            #acceptance ratio
            accept_ratio = min(1,(p_xd_num*p_theta_num*transition_num)/(p_xd_denom*p_theta_denom*transition_denom))
            print('accept_ratio =', accept_ratio)
            
            #Resampling
            u = np.random.uniform(0,1)
            if u < accept_ratio :
                theta[k,d] = theta_prop
                print('Accepted!')
                
            
            #print('k =', k,
            #    'd =', d, 
            #    'theta_kd =', theta[k,d], 
            #    'theta_prop =', theta_prop, 
            #    '; transition_num =', transition_num, 
            #    '; transition_denom =', transition_denom, 
            #    '; p_theta_num =',p_theta_num, 
            #     '; p_theta_denom =', p_theta_denom, 
            #     '; p_xd_num =', p_xd_num, 
            #     '; p_xd_denom =', p_xd_denom, 
            #     '; accept_ratio =', accept_ratio, 
            #     '; u =', u)

    

j : 0/4
Resampling
theta_kd = 5.57249149224e-07 theta_prop = 0.0
transition_num = inf
transition_denom = nan
p_theta_num = inf
p_theta_denom = 40255.4492784
p_xd_num = 1.18908606053e-267
p_xd_denom = 1.18908606053e-267
accept_ratio = 1
Accepted!
k = 0 d = 0 theta_kd = 0.0 theta_prop = 0.0 ; transition_num = inf ; transition_denom = nan ; p_theta_num = inf ; p_theta_denom = 40255.4492784 ; p_xd_num = 1.18908606053e-267 ; p_xd_denom = 1.18908606053e-267 ; accept_ratio = 1 ; u = 0.22078438443284065
Wall time: 3h 18min 45s


In [ ]:
U = np.dot(Z,Z.T)
#print(U)
U_hat = np.dot(Z_hat,Z_hat.T)
#print(U_hat)

In [ ]:
#U.shape

### 4. $U$ and $\hat{U}$ comparison

In [ ]:
fig = figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.title.set_text('U')
ax2.title.set_text('$\hat{U}$')

ax1.spy(U)
ax2.spy(U_hat)

show()